# Replica project

### Clustering efforts



In [16]:
# loading the metadata
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import pickle
import sys
import sklearn


sys.path.insert(0, "../model/")
from utils import *

sys.path.insert(0, "../web_annotation/")
from utils_clusters import *
from metrics_clusters import * 


data_dir = '/scratch/students/schaerf/'
data_dir = '../data/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
type_clustering = 'mix'
subfolder_dir = '25-05-2022'
dist = 2000
dist2 = 0.12

data_file = 'data_sample.csv'

embeds_file = subfolder_dir + '/resnext-101_'+subfolder_dir+'.npy'
map_file = subfolder_dir + '/map2pos.pkl'
cluster_file = subfolder_dir + '/clusters_'+type_clustering+'_'+str(dist)+'_'+subfolder_dir+'_19.pkl'

In [3]:
morpho = pd.read_csv(data_dir + 'morphograph_clusters.csv')
embeds = np.load(data_dir + embeds_file, allow_pickle=True) 

morpho[morpho['type'].isin(['POSITIVE', 'NEGATIVE'])].tail(1)

,img1,img2,type,uid_connection,date,cluster_file,cluster
3169,ff57569da2134eec861c70f4ac081db9,3a0213bf8ec54fea8deb92d1fa9543f8,NEGATIVE,ff57569da2134eec8deb92d1fa9543f8,26-05-2022_13:01:33,25-05-2022/clusters_mix_2000_25-05-2022_19,257


In [4]:
img1 = '05f5c4e0325f40019eeb0bb542c1f692'
img2 = '14a738ba9b91454a997738105ccf5c9b'
img3 = '5a889994248f4f67ad62057ceab2297d'

img0 = '279e34aca7474483a424b5c154719d51'
img2 = '359c34d1017e46be9f17a3b6a8a4dfb9'

# img1 = '03831e0f677d4c43ae34185065b93191'
# img2 = '5b18e18dfb5b49ceb98e0c36b1a8cec5'
# img3 = '683def2a2a19429799a6514bc7bd4156'

sklearn.metrics.pairwise.cosine_distances(embeds[embeds[:,0] == img0][0][1].reshape(1, -1), embeds[embeds[:,0] == img2][0][1].reshape(1, -1)), sklearn.metrics.pairwise.cosine_distances(embeds[embeds[:,0] == img1][0][1].reshape(1, -1), embeds[embeds[:,0] == img3][0][1].reshape(1, -1))

(array([[0.08652782]], dtype=float32), array([[0.09683412]], dtype=float32))

In [ ]:
clusters = make_clusters_embeddings(data_dir, dist=dist, data_file=data_file, embed_file=embeds_file, type_clustering=type_clustering, dist2=dist2)

In [37]:
with open(data_dir + cluster_file, 'wb') as outfile:
    pickle.dump(clusters, outfile)

In [ ]:
map2pos = get_2d_pos(data_dir, embed_file=embeds_file)
with open(data_dir + map_file, 'wb') as outfile:
    pickle.dump(map2pos, outfile)

In [11]:
positives = update_morph(data_dir, subfolder_dir)
novelty_scores = novelty_score(positives, subfolder_dir)
novelty_scores

(39983, 20)
(39983, 22)
(6517, 6)


{'original size': 4966,
 'newly added': 224,
 'additions to existing clusters': 136,
 'number of clusters with new elements': 46,
 'new clusters': 36,
 'new clusters elements': 88,
 'progress': '4.51%'}

In [12]:
metadata = pd.read_csv(data_dir + 'data_sample.csv')

In [13]:
data = pd.concat([metadata, positives], axis=0).reset_index().groupby(['uid', 'uid_connection']).last().reset_index()
print(data.shape)

(12811, 26)


In [14]:
data.to_csv(data_dir + 'data_retrain_1.csv', index=False)

In [9]:
metadata = metadata.drop(columns=['img1', 'img2', 'type', 'annotated', 'index', 'cluster', 'set', 'uid_connection'])
embeddings = np.load(data_dir + embeds_file, allow_pickle=True) 
morpho_graph_clusters = pd.read_csv(data_dir + 'morphograph_clusters.csv')
morpho_graph_clusters = morpho_graph_clusters[morpho_graph_clusters['cluster_file'].str.contains(subfolder_dir)]


with open(data_dir + 'uid2path.pkl', 'rb') as infile:
    uid2path = pickle.load(infile)

In [20]:
positives.columns

Index(['uid_connection', 'index', 'uid', 'AuthorOriginal', 'Description',
       'AuthorBirth', 'path', 'BeginDate', 'ImageURL', 'City', 'Country',
       'AuthorBirthCity', 'AuthorDeathCity', 'Drawer', 'Type', 'img1', 'img2',
       'type', 'annotated', 'cluster_file', 'cluster', 'set', 'new_cluster',
       'new set', 'old_cluster', 'C'],
      dtype='object')

In [21]:
sets = make_new_train_set(embeddings, positives, morpho_graph_clusters, subfolder_dir, uid2path)
sets

(3174, 7)


100%|██████████| 5190/5190 [05:01<00:00, 17.22it/s]


(11178, 7)
(4966, 7)
                                     A                                 B  \
5185  a7d5691e942b4f439f4fbe15679a95a8  111f79c1ec1f4bfeb70b209d98204b4d   
5185  a7d5691e942b4f439f4fbe15679a95a8  111f79c1ec1f4bfeb70b209d98204b4d   
5186  1a4ecebdd379406bb7c686121022caf5  7cafbc0aa8bf4311b87967339b46aa99   
5189  879d9e32714748b29ba97eb712ca2dfb  0c59e402e28e44848d0592dba5204d63   
5189  879d9e32714748b29ba97eb712ca2dfb  0c59e402e28e44848d0592dba5204d63   

                                     C    set  \
5185  c97fd0b8be4648a186d07469f330c6a0  train   
5185  41adde25779c412e8edcbd8caa987293  train   
5186  96f1701981d84f30ad0ddc3bd758a3ce    val   
5189  9be205855c7944918e96a93fa405e603    val   
5189  8ef7657c35d44938870e30a4e46a96a4    val   

                                                 A_path  \
5185  /mnt/project_replica/datasets/cini/53C/53C_256...   
5185  /mnt/project_replica/datasets/cini/53C/53C_256...   
5186  /mnt/project_replica/datasets/cini/58A/58A_4

,A,B,C,set,A_path,B_path,C_path
2,ae828b07e4284d80b652206ffb77278a,ea64207684694d379579998f88b7c29a,29bec596297940aeb911cb399ede86ee,test,/mnt/project_replica/datasets/cini/158C/158C_1...,/mnt/project_replica/datasets/cini/44B/44B_109...,/mnt/project_replica/datasets/cini/137B/137B_2...
3,76b2c4aeab404bc79768117e3ddc62b1,b8dd2c0c2c7146e68e5bbb0aaf6cf803,65ef07edfc9b48b49e701d169358cbd1,val,/mnt/project_replica/datasets/cini/47A/47A_858...,/mnt/project_replica/datasets/cini/61C/61C_348...,/mnt/project_replica/datasets/cini/97B/97B_341...
8,58908f6a22dd4e9cace587fb7b4ba56a,5d8f20bce78042758a75931c70aaa615,1f436fa96c154841aa0db425cbec2dad,val,/mnt/project_replica/datasets/WGA/images/c/cou...,/mnt/project_replica/datasets/WGA/images/g/gio...,/mnt/project_replica/datasets/WGA/images/c/cra...
9,406a36c78eb24b289871c117e61cdafb,3b929f6ff1ef4c5d9c5bf4b2251591c8,3ee4fc34fcd04f10aa635e8b3fbb9557,val,/mnt/project_replica/datasets/WGA/images/c/can...,/mnt/project_replica/datasets/WGA/images/c/can...,/mnt/project_replica/datasets/WGA/images/c/can...
9,406a36c78eb24b289871c117e61cdafb,3b929f6ff1ef4c5d9c5bf4b2251591c8,eff9aed3780b4fc28b35c35d30f9a5f7,val,/mnt/project_replica/datasets/WGA/images/c/can...,/mnt/project_replica/datasets/WGA/images/c/can...,/mnt/project_replica/datasets/WGA/images/c/can...
...,...,...,...,...,...,...,...
5185,a7d5691e942b4f439f4fbe15679a95a8,111f79c1ec1f4bfeb70b209d98204b4d,c97fd0b8be4648a186d07469f330c6a0,train,/mnt/project_replica/datasets/cini/53C/53C_256...,/mnt/project_replica/datasets/cini/47A/47A_824...,/mnt/project_replica/datasets/cini/115A/115A_7...
5185,a7d5691e942b4f439f4fbe15679a95a8,111f79c1ec1f4bfeb70b209d98204b4d,41adde25779c412e8edcbd8caa987293,train,/mnt/project_replica/datasets/cini/53C/53C_256...,/mnt/project_replica/datasets/cini/47A/47A_824...,/mnt/project_replica/datasets/cini/31A/31A_643...
5186,1a4ecebdd379406bb7c686121022caf5,7cafbc0aa8bf4311b87967339b46aa99,96f1701981d84f30ad0ddc3bd758a3ce,val,/mnt/project_replica/datasets/cini/58A/58A_475...,/mnt/project_replica/datasets/cini/58A/58A_30.jpg,/mnt/project_replica/datasets/cini/58A/58A_555...
5189,879d9e32714748b29ba97eb712ca2dfb,0c59e402e28e44848d0592dba5204d63,9be205855c7944918e96a93fa405e603,val,/mnt/project_replica/datasets/cini/57A/57A_795...,/mnt/project_replica/datasets/cini/122A/122A_4...,/mnt/project_replica/datasets/cini/46B/46B_243...


In [22]:
train_set = sets[sets['set'] == 'train']
val_set = sets[sets['set'] == 'val']

train_set.to_csv(data_dir + 'retrain_1_train.csv')
val_set.to_csv(data_dir + 'retrain_1_val.csv')

In [23]:
scores_clusters = cluster_accuracy(morpho_graph_clusters)
scores_clusters

0.75